# The Progress Bar!

This will contain all of my experiments along the way to deepening my understanding of the Attention mechanism and Transformers. 

The idea is to later organise this into a clean library once I have a working version (or do this in parts as working pieces of the code are implemented in this notebook playground).

I will be following Peter Bloem's blog on Transformers [here](https://peterbloem.nl/blog/transformers). I understand that this already comes with a [GitHub repository](https://github.com/pbloem/former) but I will try to implement my own version from scratch without looking up any of the code. Of course, not all resemblances will be coincidental but I'll do my best!   

I will leave it intentionally unkempt and unedited.

## Step 0: Boilerplate Imports

In [2]:
import numpy as np
from einops import *
import torch
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import nn


## Step 1: The Self-Attention

Given a $batch \times {seq\_len} \times dim$ input matrix $X$, we calculate the scaled dot product with self.

First we'll calculate the unnormalized weights $w'_{ij} = x_i^\mathbf{T}x_j$. We could naively loop over all $(i, j)$ pairs for each batch of sequence vectors but there's an easier way.

First, let's assume the simplest case when $batch = 1$. In this case we have a single sequence of vectors $X = <x_1, x_2, ..., x_n>$ each of size $dim$. If we multiply $X$ with its transpose $X^\mathbf{T}$, we'll get exactly what we want: each row of $X$ will be multiplied by the each column $X^\mathbf{T}$ which is, you guessed it, rows of $X$! Now adding the batch dimension doesn't change anything if we only permute the $seq\_len$ and $dim$ dimensions.

As it happens, torch already provides a facility for this exact situation in `torch.bmm` which stands for **Batched Matrix Multiply**. This can easily be implemented with `torch.matmul` as well.

In [7]:
def w_prime(X):
    return torch.bmm(X, X.transpose(1, 2))

In order to turn this into the scaled positive weights that sum to one, we'll have to apply a Softmax function to the **rows** of $W'$. Why? We know that in $W'$ in each batch, for each sequence, the weight of each element in the sequence is presented in the last dimension. 

Let's test out `w_prime` to see what the shape of the output is compared to the input.

In [8]:
batch_size, seq_len, dim = 32, 64, 8
X = torch.randn(batch_size, seq_len, dim)
print(X.shape)
print(w_prime(X).shape)

torch.Size([32, 64, 8])
torch.Size([32, 64, 64])


This makes sense right? In each batch, for every sequence, every element in the sequence will have a weight against every other element in the sequence, itself included. So the shape of $W'$ will be $batch \times seq\_len \times seq\_len$ where the final dimension houses the unscaled weights. If we apply a softmax to this dimension we will get the scaled weights that we want.

In [14]:
def w_scaled(X):
    unscaled_weights = w_prime(X)
    return F.softmax(unscaled_weights, dim=-1)

And for a sanity check:

In [15]:
print(w_prime(X)[0,0,:].sum().item())
print(w_scaled(X)[0,0,:].sum().item())

11.168716430664062
1.0


Now all that's left is to weight each input sequence by its scaled self-attention weight:

In [50]:
def basic_self_attention(X):
    return torch.bmm(w_scaled(X), X)

In [51]:
basic_self_attention(X).shape

torch.Size([32, 64, 8])

Note that here for each instance in a batch we have a $64 \times 64$ weight matrix (where each row corresponds to the self-attention weights for the element in that position) being multiplied by a $64 \times 8$ input matrix which ultimately results in a $32 \times 64 \times 8$ matrix where each sequence embedding is now weighted by self-attention scores.